In [43]:
import pandas as pd
from scipy import stats

### Hypotéza 1:
Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako dálnicích.
K ověření hypotézy využijte $\chi^2$ test s tím, že také určíte, jestli nehody na silnicích 1. třídy
vedly častěji či méně často k fatální nehodě. V tom vám může pomoct “expected” výstup $\chi^2$
testu. Nerozlišujte počet následků, ale pouze nehody, kde došlo k jednomu a více úmrtí
(p13a  > 0).

V načteném dataframu zachováme pouze nehody na dalnicích (0) a silnicích první třídy (1) a fatalitu nehod

In [44]:
df = pd.read_pickle("accidents.pkl.gz")[['p36', 'p13a']]
df = df[df['p36'].isin([0,1])]

Vytvoříme nový sloupec určující zda došlo k umrtí či nikoliv

In [45]:
df['death'] = False
df.loc[df['p13a'] >= 1,'death'] = True

Vytvoříme kontingenční tabulku pro zavíslost mezi umrtím a typem silnice

In [46]:
ct = pd.crosstab(df['p36'], df['death'])
ct

death,False,True
p36,,
0,24293,166
1,78618,911


provedeme $\chi^2$ test zavislosti

In [47]:
stats.chi2_contingency(ct)[1]

3.6067450279444316e-10

p-value < 0.05 -> zamítame nulovou hypotézu. Na dálnicích se umíralo s jinou pravděpodobností jako na silnici 1. třídy"

provedeme porovnáni očekávaných $\chi^2$ a skutečných hodnot

In [48]:
exp = stats.chi2_contingency(ct)[3]
print(str(exp[1,0]/exp[1,1]) + ' >? ' + str(ct.iloc[1,0]/ct.iloc[1,1]))

95.55338904363974 >? 86.29857299670691


bylo očekávano více nehod na silnici 1. tridy, tudíž vedly méně často k smrtelným nehodám

### Hypotéza 2:
Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.
Využijte vhodný test a určete, zda je škoda na vozidlech vyšší či nižší a zda je tato vlastnost
na požadované hladině významnosti.

V načteném dataframu zachováme pouze nehody požadovaných značek a jejich škody

In [49]:
df = pd.read_pickle("accidents.pkl.gz")[['p45a', 'p53']]
# 2=audi 39=skoda 47=vw
brand1, brand2 = 2, 39
df = df[(df['p45a'].isin([brand1,brand2]))] #  & (df['p53'] > 0) ]

`The Mann-Whitney U test is used to compare whether there is a difference in the dependent variable for two independent groups`
použijeme tedy mannwhitneyu neparovy test.

In [50]:
stats.mannwhitneyu(df[df['p45a'].isin([brand1])]['p53'],df[df['p45a'].isin([brand2])]['p53'])

MannwhitneyuResult(statistic=789001736.0, pvalue=4.921643600739696e-188)

p-value < 0.25 -> zamítame nulovou hypotézu. Pruměrné škody těchto značek se liší

In [51]:
print('audi')
df[df['p45a'] == brand1]['p53'].describe()

audi


count    11445.000000
mean       568.358934
std        894.719634
min          0.000000
25%        100.000000
50%        300.000000
75%        700.000000
max      20000.000000
Name: p53, dtype: float64

In [52]:
print('skoda')
df[df['p45a'] == brand2]['p53'].describe()

skoda


count    118379.000000
mean        368.166026
std         471.395447
min           0.000000
25%         100.000000
50%         200.000000
75%         500.000000
max       15000.000000
Name: p53, dtype: float64

audi má vyrazně vyšší průměrnou škodu cca 56800 Kč (vs cca 36800 Kč u značky škoda)